In [1]:
import warnings
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross, MeanReversionLongOnly
from src.utils import load_data

warnings.filterwarnings('ignore')
#pd.set_option('display.float_format', '{:,.2f}'.format)

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
ticker = "^SPX"
period = "max"
stock = yf.Ticker(ticker.upper())
data = stock.history(period=period)

In [3]:
data.dropna()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-10-14 00:00:00-04:00,5829.810059,5871.410156,5829.569824,5859.850098,3005250000,0.0,0.0
2024-10-15 00:00:00-04:00,5866.740234,5870.359863,5804.479980,5815.259766,3882120000,0.0,0.0
2024-10-16 00:00:00-04:00,5816.580078,5846.520020,5808.339844,5842.470215,3467230000,0.0,0.0


In [4]:
strategy = SmaCross
strategy.n1 = 42
strategy.n2 = 242
bt_sma = Backtest(data["2019":], SmaCross, cash=10_000, commission=0.004, trade_on_close=True)
stats = bt_sma.run()
stats

Start                     2019-01-02 00:00...
End                       2024-10-18 00:00...
Duration                   2115 days 23:00:00
Exposure Time [%]                   57.808219
Equity Final [$]                 19256.675989
Equity Peak [$]                  19311.815638
Return [%]                           92.56676
Buy & Hold Return [%]              133.649393
Return (Ann.) [%]                   11.974599
Volatility (Ann.) [%]               10.976313
Sharpe Ratio                         1.090949
Sortino Ratio                        1.784646
Calmar Ratio                         0.905471
Max. Drawdown [%]                  -13.224718
Avg. Drawdown [%]                   -1.366578
Max. Drawdown Duration      561 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      46.136293
Worst Trade [%]                     41.089987
Avg. Trade [%]                    

In [5]:
bt_sma.plot()

Row(id='1450', ...)

In [6]:
%%time
stats = bt_sma.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: user 3.98 s, sys: 76.3 ms, total: 4.06 s
Wall time: 4.06 s


Start                     2019-01-02 00:00...
End                       2024-10-18 00:00...
Duration                   2115 days 23:00:00
Exposure Time [%]                   67.945205
Equity Final [$]                 18178.242869
Equity Peak [$]                  18811.829293
Return [%]                          81.782429
Buy & Hold Return [%]              133.649393
Return (Ann.) [%]                   10.866253
Volatility (Ann.) [%]               11.910193
Sharpe Ratio                         0.912349
Sortino Ratio                        1.497408
Calmar Ratio                         0.692615
Max. Drawdown [%]                  -15.688741
Avg. Drawdown [%]                   -1.640685
Max. Drawdown Duration      395 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                   34
Win Rate [%]                        67.647059
Best Trade [%]                      24.609156
Worst Trade [%]                     -9.618692
Avg. Trade [%]                    

In [7]:
stats._strategy

<Strategy SmaCross(n1=5,n2=20)>

In [8]:
bt_sma.plot()

Row(id='2182', ...)

In [9]:
%%time
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=False)
stats = bt_momentum.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

CPU times: user 10.1 s, sys: 60.2 ms, total: 10.1 s
Wall time: 10.2 s


Start                     1927-12-30 00:00...
End                       2024-10-18 00:00...
Duration                  35356 days 23:00:00
Exposure Time [%]                    1.089773
Equity Final [$]                     5.140937
Equity Peak [$]                       10000.0
Return [%]                         -99.948591
Buy & Hold Return [%]            33108.776741
Return (Ann.) [%]                   -7.548037
Volatility (Ann.) [%]                5.084744
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.949791
Avg. Drawdown [%]                  -99.949791
Max. Drawdown Duration    35336 days 00:00:00
Avg. Drawdown Duration    35336 days 00:00:00
# Trades                                  222
Win Rate [%]                         1.801802
Best Trade [%]                        3.41795
Worst Trade [%]                    -10.477453
Avg. Trade [%]                    

In [10]:
bt_momentum.plot()

TypeError: Index.get_loc() got an unexpected keyword argument 'method'

In [ ]:
bt_momentum = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0.04, trade_on_close=True)
stats = bt_momentum.run()
stats